# Task 1

## Loading Libraries & Dataframe

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import missingno as msno
import re

from scipy import stats
from scipy.stats import skew, boxcox, mstats
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
path_df = pd.read_csv(r"laptop_prices.csv")
path_df

## A quick overview **(EDA)**

In [ ]:
path_df.columns

In [ ]:
path_df.info()

No missing values in the dataset

In [ ]:
msno.matrix(path_df)

No duplicates in dataset

In [ ]:
path_df.duplicated().sum()

### ~`Company`
Column seems to have a high cardinality between it's results due to companies having bigger market shares

In [ ]:
company_names = path_df["Company"].unique()
print(f"Laptop companies: {company_names}")

plt.figure(figsize= (18,12))

company_counts = path_df["Company"].value_counts()
avg_company_count = company_counts.mean()

# List comprehension for color condition
colors = ["green" if count >= avg_company_count else "red" for count in company_counts]

sns.countplot(x = "Company",
              data = path_df,
              order = company_counts.index,
              palette = colors,
              edgecolor = "black")
plt.title("Market shares of laptop companies")
# Average count line
plt.axhline(y = avg_company_count,
            color = "black",
            linestyle = "--",
            label = f"Mean: {avg_company_count:.0f}")
plt.legend()
plt.show()

### `Product`
Column has repeated entries (nothing concerning)

In [ ]:
path_df["Product"].nunique()

### ~`TypeName`
**Notebooks** seem to be the most popular

In [ ]:
unique_types = path_df["TypeName"].nunique()
print(f"Laptop types: {unique_types}")

plt.figure(figsize = (18,12))

typecounts = path_df["TypeName"].value_counts()

sns.countplot(x = "TypeName",
              data = path_df,
              palette = "Set2",
              edgecolor = "black",
              order = typecounts.index)
plt.title("Laptop type popularity")
plt.xlabel("Laptop Type")
plt.show()

### `Inches`
Column seemed to have multiple high leverage points that after a deep research it was concluded that they weren't outliers. They were small sized due to them mostly being convertibles or small netbooks

In [ ]:
avg_size = path_df["Inches"].mean()
print(f"Average laptop size: {avg_size:.0f} inches")

Q1 = path_df["Inches"].quantile(0.25)
Q3 = path_df["Inches"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = path_df[(path_df["Inches"] < lower) | (path_df["Inches"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (18,12))
sns.boxplot(x = "Inches",
            data = path_df,
            color = "grey")
plt.show()

### `Ram`
Just like last column. `Ram` Also had high leverage points in terms of ram capacity beihng above the average in some cases. That was due to these laptops being a high performance **gaming** or **workstation** laptops

In [ ]:
avg_ram = path_df["Ram"].mean()
print(f"Average laptop ram: {avg_ram:.0f} GB")

Q1 = path_df["Ram"].quantile(0.25)
Q3 = path_df["Ram"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = path_df[(path_df["Ram"] < lower) | (path_df["Ram"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (18,12))
sns.boxplot(x = "Ram",
            data = path_df,
            color = "grey")
plt.show()

### ~`OS`
We see high cardinality of categorical variables with a large dominance of windows 10 being the main OS for most laptops

In [ ]:
os_counts = path_df["OS"].value_counts()
os_avg = os_counts.mean()
os_num = path_df["OS"].nunique()
print(f"There are a total of {os_num} operating systems in the market")

color = ["green" if count > os_avg else "red" for count in os_counts]

plt.figure(figsize = (18,12))
sns.countplot(x = "OS",
              data = path_df,
              palette = color,
              order = os_counts.index,
              edgecolor = "black")
plt.axhline(y = os_avg,
            color = "black",
            linestyle = "--",
            label = f"Mean: {os_avg:.0f}")
plt.legend()
plt.title("Preinstalled operating systems in laptops")
plt.show()

### `Weight`
There seems to be a couple influencial entires where they weigh more than 2.71 KG but it's to be expected due to their bigger surface area compared to other laptops

In [ ]:
avg_weight = path_df["Weight"].mean()
print(f"Average weight: {avg_weight:.2f} KG")

Q1 = path_df["Weight"].quantile(0.25)
Q3 = path_df["Weight"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q1 + 1.5 * IQR

outliers = path_df[(path_df["Weight"] < lower) | (path_df["Weight"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (18,12))
sns.boxplot(x = "Weight",
            data = path_df,
            color = "grey")
plt.show()

As we see, there's a positive linear relationship between the laptop size in inches and its weight

In [ ]:
plt.figure(figsize = (12,10))
sns.regplot(x = "Inches",
                y = "Weight",
                data = path_df,
                scatter_kws = {"color" : "grey", "alpha" : 0.8},
                line_kws = {"color" : "red", "linewidth" : 2})
plt.title("Relationship between laptop size & weight")
plt.show()

### `Price_euros`
Price seems to be skewed which will need transformation to adjust and normalize or checking outliers

In [ ]:
def state_skew(x):
    if abs(x) > 1:
        return "Very skewed"
    elif 0.5 <= abs(x) <= 1:
        return "Moderately skewed"
    else : 
        return "Symmetrical"
def state_kurtosis(x):
    if x > 1:
        return "Leptokurtic (Many Outliers)"
    elif x < -1:
        return "Playtukurtic (Few Outliers)"
    else :
        return "Mesokurtic (Normal tails)"
    
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (12, 10))
sns.histplot(x = "Price_euros",
             data = path_df,
             color = "grey",
             edgecolor = "black",
             kde = True,
             ax = ax1)
ax1.set_title("Checking column normality")
sns.regplot(x = "Price_euros",
                y = "Weight",
                data = path_df,
                scatter_kws = {"color" : "grey", "alpha" : 0.8},
                line_kws = {"color" : "red", "linewidth" : 2},
                ax = ax2)
ax2.set_title("Relationship of price with weight")
plt.show()

price_kurtosis = path_df["Price_euros"].kurtosis()
price_skew = path_df["Price_euros"].skew()
print(f"Price kurtosis before transformation: {price_kurtosis:.2f}, {state_kurtosis(price_kurtosis)}")
print(f"Price skew before transformation: {price_skew:.2f}, {state_skew(price_skew)}")

From what we can see is there are multiple outliers that could affect the modeling phase.
After further investigation they could seem dangerous but after transformation most likely it'll be completely normalized due to these highly priced laptops being either workstation, ultrabooks or gaming laptops that have alot of expensive parts

In [ ]:
avg_price = path_df["Price_euros"].mean()
print(f"Average price: {avg_price:.2f}€")

Q1 = path_df["Price_euros"].quantile(0.25)
Q3 = path_df["Price_euros"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q1 + 1.5 * IQR

outliers = path_df[(path_df["Price_euros"] < lower) | (path_df["Price_euros"] > upper)]
print(f"Number of outliers: {len(outliers)}")
print(f"Outlier bounds: [{lower:.2f}, {upper:.2f}]")
display(outliers)

plt.figure(figsize = (12, 10))
sns.boxplot(x = "Price_euros",
            data = path_df,
            color = "grey")
plt.show()

### ~`Screen`, `ScreenW`, & `ScreemH`
Dominance of **Full HD** & **Standard** over the laptop market (High Cardinality)

In [ ]:
screen_count = path_df["Screen"].value_counts()

plt.figure(figsize = (10,8))
plt.pie(screen_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%",
        explode = [0, 0.1, 0, 0])  # Shows percentage
plt.title("Laptop screens by popularity")
plt.legend(labels = screen_count.index,
           title = "Screen type",
           loc = "best")
plt.show()

**1920x1080p** dominated laptop screens and to this day it's the most popular and most used resolution although **2560x1440p** & **3840x2160p** (also known as **4K**) has been getting more popular in the recent years

In [ ]:
# Generating the full screen resolution
path_df["Resolution"] = path_df["ScreenW"].astype(str) + "x" + path_df["ScreenH"].astype(str)
res_count = path_df["Resolution"].value_counts()
avg_res = res_count.mean()

color = ["green" if count >= avg_res else "red" for count in res_count]

plt.figure(figsize = (18, 12))
sns.countplot(x = "Resolution",
              data = path_df,
              order = res_count.index,
              palette = color,
              edgecolor = "black")
plt.title("Most popular laptop resolutions")
plt.axhline(y = avg_res,
            color = "black",
            linestyle = "--",
            label = f"Mean: {avg_res:.0f}")
plt.legend()
plt.show()

### ~`Touchscreen`

In [ ]:
touch_count = path_df["Touchscreen"].value_counts()

plt.figure(figsize = (10, 8))
plt.pie(touch_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%",
        explode = [0, 0.1])
plt.title("Touchscreen laptop proportion")
plt.legend(labels = touch_count.index,
           title = "Touchscreen ?",
           loc = "best")

### ~`IPSpanel`

In [ ]:
ips_count = path_df["IPSpanel"].value_counts()

plt.figure(figsize = (10, 8))
plt.pie(ips_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%",
        explode = [0, 0.1])
plt.title("IPS screens laptop proportion")
plt.legend(labels = ips_count.index,
           title = "IPS ?",
           loc = "best")

### ~`RetinaDisplay`
Very little laptops had retina display screens where this technology was mostly unique to apple macbooks

In [ ]:
retina_count = path_df["RetinaDisplay"].value_counts()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (10,8))
ax2.pie(retina_count.values,
        colors = sns.color_palette("Set2"),
        autopct = "%1.1f%%")
ax2.legend(labels = retina_count.index,
           title = "Retina ?",
           loc = "upper right")
ax2.set_title("Retina display laptop proportion")

# Showing only companies making retina display laptops
retina_by_company = path_df[path_df["RetinaDisplay"] == "Yes"].groupby("Company")["RetinaDisplay"].count().sort_values(ascending = False)
retina_companies = retina_by_company[retina_by_company > 0]

sns.barplot(x = retina_companies.index,
            y = retina_companies.values,
            color = "grey",
            edgecolor = "black",
            ax = ax1)
ax1.set_title("Retina display by company")
ax1.tick_params(axis = "x", rotation = 45)
plt.tight_layout()
plt.show()

print(f"Only {retina_companies.shape[0]} companies have retina display laptops")
print(f"Total Retina laptops: {retina_count.get(1, 0)} ({retina_count.get(1, 0)/len(path_df)*100:.1f}%)")

### ~`CPU_company`
Intel holds a massive market share in CPUs for laptops

In [ ]:
plt.figure(figsize = (12, 10))
sns.countplot(x = "CPU_company",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Laptop cpu companies market shares")
plt.show()

### `CPU_freq`
Although **Intel** holds the bigger marketshare. **AMD** seems to have the better performing CPUs on average

In [ ]:
cpu_company_freq = path_df.groupby("CPU_company")["CPU_freq"].mean().sort_values(ascending = False).round(1)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (12, 10))
sns.histplot(x = "CPU_freq",
             data = path_df,
             color = "grey",
             edgecolor = "black",
             kde = True,
             ax = ax1)
ax1.set_title("Laptop CPU clock speed distribution")
ax1.set_ylabel("Frequency (GHz)")

sns.barplot(x = cpu_company_freq.index,
            y = cpu_company_freq.values,
            palette = "Set2",
            edgecolor = "black",
            ax = ax2)
ax2.set_title("Average CPU clock speed per company")
ax2.set_ylabel("Frequency (GHz)")
for i, v in enumerate(cpu_company_freq.values):
    ax2.text(i, v - 0.15, f"{v:.1f}GHz", ha = "center", fontweight = "bold")
plt.show()

### ~`CPU_model`
There is a massive popularity of "**Core i**" models compared to other models of **Intel** and other CPU companies (High Cardinality)

In [ ]:
# Extracting 
def extract_cpu_model(cpu_string):
    patterns = [
        r'(Core [im]\d)',           # Core i3, i5, i7, i9, m3, m5, etc...
        r'(Ryzen \d)',              # Ryzen 3, 5, 7
        r'(Atom [xX]\d)',           # Atom x5, x7
        r'(Celeron)',               # Celeron
        r'(Pentium)',               # Pentium
        r'(Xeon)',                  # Xeon
        r'(A\d+)',                  # AMD A6, A9, A10, A12, etc...
        r'(E2)',                    # AMD E2
        r'(FX)',                    # AMD FX
    ]
    
    for pattern in patterns:
        match = re.search(pattern, cpu_string, re.IGNORECASE)
        if match:
            return match.group(1)
    return "Other"

path_df["CPU_series"] = path_df["CPU_model"].apply(extract_cpu_model)
series_counts = path_df["CPU_series"].value_counts()
avg_series = series_counts.mean()

colors = ["green" if count > avg_series else "red" for count in series_counts]

plt.figure(figsize = (16,10))
sns.countplot(x = "CPU_series",
              data = path_df,
              palette = colors,
              order = series_counts.index,
              edgecolor = "black")
plt.axhline(y = avg_series,
            color = "black",
            linestyle = "--",
            label =f"Mean: {avg_series:.0f}")
plt.legend()
plt.title("CPU models popularity in laptops")
plt.show()

### `PrimaryStorage`
Most laptops had **256GBs** as a primary drive that had their operating system on

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "PrimaryStorage",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Primary storage popularity in GBs")
plt.xlabel("GBs")
plt.show()

### `SecondaryStorage`
Most laptops don't have any secondary hard drives installed which could result in an imbalance in modeling later on

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "SecondaryStorage",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Secondary storage popularity in GBs")
plt.xlabel("GBs")
plt.show()

### ~`PrimaryStorageType`
**SSDs** generally have been the most popular storage type due to their fast read and write frequency & longevity

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "PrimaryStorageType",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Primary storage type popularity")
plt.xlabel("Storage Type")
plt.show()

### ~`SecondaryStorageType`
Surprisingly laptops that do have a seconadry storage installed usually use **HDD** which means that usually use secondary storage to store important files and don't need to use the massive read & write speeds that an **SSD** provides

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "SecondaryStorageType",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Secondary storage type popularity")
plt.xlabel("Storage Type")
plt.show()

### ~`GPU_company`
**Intel** has the largest market share which is expected due to them winning by a landslide in CPUs as well

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(x = "GPU_company",
              data = path_df,
              palette = "Set2",
              edgecolor = "black")
plt.title("Laptop GPU companies market share")
plt.xlabel("GPU Companies")
plt.show()

### ~`GPU_model`
**Intel integrated graphics** had the highest market share due to them being present in most budget to medium priced laptop without a dedicated GPU

In [ ]:
# Extracting GPU series
def extract_gpu_model(gpu_string):
    gpu_string = str(gpu_string)
    
    # NVIDIA patterns
    if re.search(r'GTX', gpu_string, re.IGNORECASE):                    # GTX
        return "GeForce GTX"
    elif re.search(r'MX\d{3}', gpu_string, re.IGNORECASE):              # MX
        return "GeForce MX"
    elif re.search(r'GeForce \d{3}', gpu_string, re.IGNORECASE):        # GeForce
        return "GeForce (Low-end)"
    elif re.search(r'Quadro', gpu_string, re.IGNORECASE):               # Quadro
        return "Quadro"
    
    # AMD patterns
    elif re.search(r'Radeon Pro', gpu_string, re.IGNORECASE):           # Radeon Pro
        return "Radeon Pro"
    elif re.search(r'Radeon RX', gpu_string, re.IGNORECASE):            # Radeon RX
        return "Radeon RX"
    elif re.search(r'Radeon R\d', gpu_string, re.IGNORECASE):           # Radeon R
        return "Radeon R"
    elif re.search(r'Radeon', gpu_string, re.IGNORECASE):               # Radeon 530, etc...
        return "Radeon (Other)"
    
    # Intel patterns
    elif re.search(r'UHD Graphics', gpu_string, re.IGNORECASE):         # UHD Graphics 550, etc...
        return "Intel UHD"
    elif re.search(r'Iris Plus', gpu_string, re.IGNORECASE):            # Iris
        return "Intel Iris Plus"
    elif re.search(r'Iris Pro', gpu_string, re.IGNORECASE):             # Iris Pro
        return "Intel Iris Pro"
    elif re.search(r'Iris', gpu_string, re.IGNORECASE):                 # Iris
        return "Intel Iris"
    elif re.search(r'HD Graphics', gpu_string, re.IGNORECASE):          # HD Graphics
        return "Intel HD"
    
    return "Other"

path_df["GPU_series"] = path_df["GPU_model"].apply(extract_gpu_model)
gpu_series_counts = path_df["GPU_series"].value_counts()
avg_gpu_series = gpu_series_counts.mean()

colors = ["green" if count > avg_gpu_series else "red" for count in gpu_series_counts]

plt.figure(figsize = (12, 10))
sns.countplot(x = "GPU_series",
              data = path_df,
              palette = colors,
              order = gpu_series_counts.index,
              edgecolor = "black")
plt.axhline(y = avg_gpu_series,
            color = "black",
            linestyle = "--",
            label = f"Mean: {avg_gpu_series:.0f}")
plt.legend()
plt.title("GPU series popularity in laptops")
plt.xlabel("GPU Series")
plt.xticks(rotation = 45, ha = "right")
plt.tight_layout()
plt.show()

## Choosing the variables (Feature Engineering)

### `Price_euros` **Log Transformation**
Using the log transformation has successfully normalized the column which made it suitable for modeling

In [ ]:
path_df["Price_transformed"] = path_df["Price_euros"].apply(np.log).round(2)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (12, 10))
sns.histplot(x = "Price_transformed",
             data = path_df,
             color = "grey",
             edgecolor = "black",
             kde = True,
             ax = ax1)
ax1.set_title("Checking column normality")
sns.regplot(x = "Price_transformed",
                y = "Weight",
                data = path_df,
                scatter_kws = {"color" : "grey", "alpha" : 0.8},
                line_kws = {"color" : "red", "linewidth" : 2},
                ax = ax2)
ax2.set_title("Relationship of price with weight")
plt.show()

price_kurtosis = path_df["Price_transformed"].kurtosis()
price_skew = path_df["Price_transformed"].skew()
print(f"Price kurtosis before transformation: {price_kurtosis:.2f}, {state_kurtosis(price_kurtosis)}")
print(f"Price skew before transformation: {price_skew:.2f}, {state_skew(price_skew)}")

### `Company`, `CPU_series` `GPU_series` **TE with smoothing**
Due to their high cardinality and imbalance we will use target encoding to reduce overfitting and smoothing to reduce data leakage.
Reducing cardinality by reducing the number of insignificant variables in these 3 columns

In [ ]:
others = {"Company": company_counts,
          "CPU_series": series_counts,
          "GPU_series": gpu_series_counts}

others_avg = {"Company": avg_company_count,
          "CPU_series": avg_series,
          "GPU_series": avg_gpu_series}

# Function that returns variable names that are less than the average EXCLUDING Apple due to having a neiche share of the market
def to_other(vals, avg):
    hitlist = vals.values < avg
    red = vals.index[hitlist]
    red = [name for name in red if name != "Apple"]
    return red
    
for col, v in others.items():
    other_vars = to_other(v, others_avg[col])
    path_df[col] = path_df[col].replace(other_vars, "Other")
    
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (18,8))
sns.countplot(x = "Company",
             data = path_df,
             palette = "Set2",
             edgecolor = "black",
             ax = ax1)
ax1.set_title("Company column after reducing cardinality")

sns.countplot(x = "CPU_series",
             data = path_df,
             palette = "Set2",
             edgecolor = "black",
             ax = ax2)
ax2.set_title("CPU_series column after reducing cardinality")

plt.figure(figsize = (18,8))
sns.countplot(x = "GPU_series",
             data = path_df,
             palette = "Set2",
             edgecolor = "black")
plt.title("GPU_series column after reducing cardinality")
plt.show()

Target encoding using kfold on 5 folds to reduce data leakage

In [ ]:
# 5 Folds
encoder = TargetEncoder(cv = 5)

te_cols = ["Company", "CPU_series", "GPU_series"]

# Setting up the tranformer
ct = ColumnTransformer(transformers = [
    ("target_enc", encoder, te_cols)
], remainder = "passthrough")

# To turn the output into a dataframe not array
ct.set_output(transform = "pandas")

X = path_df.drop(["Price_euros","Price_transformed"],axis = 1)
y = path_df["Price_transformed"]
encoded_data = ct.fit_transform(X, y)

path_df["Company_encoded"] = encoded_data["target_enc__Company"].round(2)
path_df["CPU_series_encoded"] = encoded_data["target_enc__CPU_series"].round(2)
path_df["GPU_series_encoded"] = encoded_data["target_enc__GPU_series"].round(2)
path_df[["Company_encoded","GPU_series_encoded","CPU_series_encoded"]]

### `TypeName`, `OS`, `Screen`, `CPU_company`, `PrimaryStorageType`, `SecondaryStorageType`, `GPU_company` **OHE**

In [ ]:
ohe_cols = ["TypeName", "OS", "Screen", "CPU_company", "PrimaryStorageType", "SecondaryStorageType", "GPU_company"]

# Safety check incase of new data
ohe = OneHotEncoder(handle_unknown = "ignore", sparse_output = False, drop = "first")

ct = ColumnTransformer(transformers = [
    ("ohe", ohe, ohe_cols)
], remainder = "passthrough")

ct.set_output(transform = "pandas")

encoded_data = ct.fit_transform(path_df)

rename_mappings = {
    col: col.replace("ohe__", "").replace(" ", "_") 
    for col in encoded_data.columns if col.startswith("ohe__")
}

renamed_cols = encoded_data[rename_mappings.keys()].rename(columns = rename_mappings)
path_df = pd.concat([path_df, renamed_cols], axis = 1)
path_df[renamed_cols.columns] = path_df[renamed_cols.columns].astype(int)

path_df[renamed_cols.columns]

### `Touchscreen`, `IPSpanel`, `RetinaDisplay` **BE (Binary Encoding)**

In [ ]:
binary_cols = ["Touchscreen", "IPSpanel", "RetinaDisplay"]

ord_enc = OrdinalEncoder(dtype= int)

encoded_names = [f"{col}_encoded" for col in binary_cols]

path_df[encoded_names] = ord_enc.fit_transform(path_df[binary_cols])

path_df[encoded_names]

### `ScreenW`, `ScreenH` **Interaction generation**

In [ ]:
path_df["Pixels"] = (path_df["ScreenW"] * path_df["ScreenH"]).astype(int)

plt.figure(figsize = (12,8))
sns.histplot(x = "Pixels",
            data = path_df,
            color = "grey",
            edgecolor = "black",
            bins = 20)
plt.title("Pixels distribution")


path_df["Pixels"] = path_df["Pixels"].apply(np.log1p)

plt.figure(figsize = (12,8))
sns.histplot(x = "Pixels",
            data = path_df,
            color = "grey",
            edgecolor = "black",
            bins = 20)
plt.title("Pixels distribution (Logged)")
plt.show()

### `Inches`, `Ram`, `Weight`, `PrimaryStorage`, `SecondaryStorage`

In [ ]:
numerical_col_hist = ["Inches", "Ram", "Weight", "PrimaryStorage", "SecondaryStorage", "Pixels"]

for cols in numerical_col_hist:
    plt.figure(figsize = (12,8))
    sns.histplot(x = cols,
                 data = path_df,
                 color = "grey",
                 edgecolor = "black",
                 bins = 20)
    plt.title(f"{cols} distribution")
    plt.show()
    
path_df["PrimaryStorage"] = path_df["PrimaryStorage"].apply(np.log1p)
path_df["SecondaryStorage"] = path_df["SecondaryStorage"].apply(np.log1p)

### Final Touches before modeling

In [ ]:
robust_features = ['Ram', 'PrimaryStorage', 'SecondaryStorage']
standard_features = ['Inches', 'CPU_freq', 'Pixels', "Weight"]
exclude_features = ["Price_euros", "Price_transformed", "ScreenW", "ScreenH"]

numerical_cols = path_df.select_dtypes(include = "number").drop(columns = exclude_features)

# Task 2

## Predictive Linear Regression

### Setting up data for modeling

In [ ]:
X = numerical_cols
y = path_df["Price_transformed"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 25)

print(f"Training on {X_train.shape[0]} laptops.")
print(f"Testing on {X_test.shape[0]} laptops.")

In [ ]:
X_train.columns

### VIF
To check for multicollinearity

In [ ]:
scaler = ColumnTransformer(transformers = [
    ("rob", RobustScaler(), robust_features),
    ("std", StandardScaler(), standard_features)
], remainder = "passthrough")

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_vif = pd.DataFrame()
X_vif["Features"] = X.columns
X_vif["VIF"] = [variance_inflation_factor(X_train_scaled, i) for i in range(X_train_scaled.shape[1])]

display(X_vif.sort_values(by="VIF", ascending=False))

### ElasticNet
Chose elasticnet instead of a regular linear regression due to lasso and ridge regression regularization **(coefficient punishment)**

In [ ]:
en_model = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], cv = 5, random_state = 5)

en_model.fit(X_train_scaled, y_train),
y_pred_log_en = en_model.predict(X_test_scaled)

y_pred_euros = np.expm1(y_pred_log_en)
y_actual_euros = np.expm1(y_pred_log_en)

print(f"Best L1 Ratio: {en_model.l1_ratio_}")
print(f"Best Alpha: {en_model.alpha_:.4f}")
print(f"R2 Score: {r2_score(y_test, y_pred_log_en):.2f}")
print(f"MAE: {mean_squared_error(y_test, y_pred_log_en):.2f}")

### Linearity Assumptions
The data passes most assumptions (with slight deviation in QQPlot) but overall it's solid

In [ ]:
lin_model = LinearRegression()
lin_model.fit(X_train_scaled, y_train)

y_pred = lin_model.predict(X_test_scaled)
residuals = y_test - y_pred

# =================================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Linearity
axes[0, 0].scatter(y_pred, residuals, alpha=0.5, color="grey", edgecolor="black")
axes[0, 0].axhline(y=0, color="red", linestyle="--", linewidth=2)
axes[0, 0].set_xlabel("Predicted Values")
axes[0, 0].set_ylabel("Residuals")
axes[0, 0].set_title("1. Linearity: Residuals vs Predicted")

# 2. Normality of Residuals
sm.qqplot(residuals, line="45", ax=axes[0, 1], markerfacecolor="grey", markeredgecolor="black")
axes[0, 1].set_title("2. Normality: Q-Q Plot of Residuals")

# 3. Homoscedasticity
standardized_residuals = (residuals - residuals.mean()) / residuals.std()
axes[1, 0].scatter(y_pred, np.sqrt(np.abs(standardized_residuals)), alpha=0.5, color="grey", edgecolor="black")
axes[1, 0].set_xlabel("Predicted Values")
axes[1, 0].set_ylabel("√|Standardized Residuals|")
axes[1, 0].set_title("3. Homoscedasticity: Scale-Location Plot")

# 4. Residuals Distribution
sns.histplot(residuals, kde=True, color="grey", edgecolor="black", ax=axes[1, 1])
axes[1, 1].axvline(x=0, color="red", linestyle="--", linewidth=2)
axes[1, 1].set_title("4. Residuals Distribution")

plt.tight_layout()
plt.show()

# ================================================================================

# Shapiro-Wilk Test for Normality
sample_residuals = residuals.sample(min(500, len(residuals)), random_state=42) if len(residuals) > 500 else residuals
shapiro_stat, shapiro_p = stats.shapiro(sample_residuals)
print(f"\n{'='*50}")
print("ASSUMPTION TEST RESULTS")
print(f"{'='*50}")
print(f"\n1. Normality (Shapiro-Wilk Test):")
print(f"   Statistic: {shapiro_stat:.4f}, p-value: {shapiro_p:.4f}")
print(f"   Result: {'✓ Normal' if shapiro_p > 0.05 else '✗ Not Normal'} (α=0.05)")

# Durbin-Watson Test for autocorrelation
dw_stat = durbin_watson(residuals)
print(f"\n2. Independence (Durbin-Watson Test):")
print(f"   Statistic: {dw_stat:.4f}")
print(f"   Result: {'✓ No autocorrelation' if 1.5 < dw_stat < 2.5 else '✗ Possible autocorrelation'} (ideal: ~2)")

# Breusch-Pagan Test for Homoscedasticity
X_test_with_const = sm.add_constant(X_test_scaled)
bp_stat, bp_p, _, _ = het_breuschpagan(residuals, X_test_with_const)
print(f"\n3. Homoscedasticity (Breusch-Pagan Test):")
print(f"   Statistic: {bp_stat:.4f}, p-value: {bp_p:.4f}")
print(f"   Result: {'✓ Homoscedastic' if bp_p > 0.05 else '✗ Heteroscedastic'} (α=0.05)")

# Model Performance
print(f"\n{'='*50}")
print("Model Performance")
print(f"{'='*50}")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"Mean Residual: {residuals.mean():.6f} (should be ~0)")

## Random Forest Regression